<div class="alert alert-block" style="border: 1px solid #455A64;background-color:#ECEFF1;">
본 자료 및 영상 컨텐츠는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하는 것을 금지합니다. 특히 자료에 대해서는 저작권법을 엄격하게 적용하겠습니다.
</div>

### 1. Scrapy: 여러 페이지 크롤링 방법 

> Scrapy 프로젝트는 여러 컴포넌트로 구성되며, 주요 구성 요소는 다음과 같습니다

- **Spider**: 웹 사이트를 크롤링하고, 페이지에서 데이터를 추출하는 규칙을 정의합니다.
- **Item**: 크롤링된 데이터의 구조(필드)를 정의합니다.
- **Item Pipeline**: 추출된 데이터를 처리하고 저장하는 방법을 정의합니다.
- **Settings**: 프로젝트의 설정(크롤링 정책, 지연 시간, 파이프라인 활성화 등)을 정의합니다.


### 주요 문법

- Scrapy를 사용하여 여러 페이지를 크롤링하는 경우, Spider 에서 다음과 같이 작성
  - `start_requests` 메소드를 통해 시작 URL을 동적으로 생성하고, 
  - `parse` 메소드 내에서 페이지별 데이터를 추출하며, 
  - 필요에 따라 추가적인 페이지 요청을 생성하여 크롤링을 할 수 있음

#### 1. **start_requests 메소드**

- 크롤링을 시작할 URL 목록을 정의합니다.
- `start_urls` 속성 대신 `start_requests` 메소드를 사용하여 좀 더 복잡한 로직으로 시작 URL을 생성할 수 있습니다.

```python
def start_requests(self):
    start_urls = ['http://example.com/page1', 'http://example.com/page2']
    for url in start_urls:
        yield scrapy.Request(url=url, callback=self.parse)
```

#### 2. **Request 객체**

- `scrapy.Request`: 지정된 URL로 HTTP 요청을 생성합니다. `callback` 매개변수를 통해 요청에 대한 응답을 처리할 함수를 지정할 수 있습니다.

```python
yield scrapy.Request(url='http://example.com', callback=self.parse_page)
```

#### 3. **동적 URL 생성**

- 페이지 번호나 쿼리 매개변수가 변하는 URL을 크롤링할 때는 반복문과 문자열 포매팅을 사용하여 URL을 동적으로 생성할 수 있습니다.

```python
for i in range(1, 6):
    yield scrapy.Request(url=f'http://example.com/page{i}', callback=self.parse)
```

#### 4. **parse 메소드**

- `parse` 메소드는 Scrapy가 HTTP 요청에 대한 응답을 받았을 때 호출하는 기본 콜백 함수입니다.
- 응답 데이터에서 필요한 정보를 추출하거나, 추가 요청을 생성할 수 있습니다.

```python
def parse(self, response):
    # 페이지에서 데이터 추출
    # 예: 제목 추출
    title = response.css('h1::text').get()
    yield {'title': title}
```

#### 5. **응답 객체(response)**

- Scrapy는 요청에 대한 응답을 `Response` 객체로 제공합니다.
- `response.css`: CSS 선택자를 사용하여 요소를 선택합니다.
- `response.xpath`: XPath 선택자를 사용하여 요소를 선택합니다.
- `.get()`: 선택한 요소의 첫 번째 텍스트를 반환합니다.
- `.getall()`: 선택한 모든 요소의 텍스트를 리스트로 반환합니다.


### 참고: Generator 와 yield
네, 제너레이터와 `yield` 키워드에 대해 좀 더 자세히 설명드리겠습니다.

1. 제너레이터(Generator):
   - 제너레이터는 값을 한 번에 하나씩 생성하는 특별한 종류의 함수입니다.
   - 일반 함수는 `return` 문을 사용하여 값을 반환하고 함수를 종료하지만, 제너레이터는 `yield` 문을 사용하여 값을 반환하고 함수의 상태를 유지합니다.
   - 제너레이터는 `next()` 함수를 호출할 때마다 다음 `yield` 문까지 실행되고, 해당 값을 반환합니다.
   - 제너레이터는 모든 값을 한 번에 생성하지 않고, 필요할 때마다 값을 생성하므로 메모리 효율적입니다.

2. `yield` 키워드:
   - `yield` 키워드는 제너레이터 함수 내에서 사용되며, 값을 반환하지만 함수를 종료하지 않습니다.
   - `yield` 문을 만나면 해당 값을 반환하고, 함수의 실행 상태를 유지한 채로 일시 중단됩니다.
   - 다음 `next()` 호출 시, 함수는 이전에 중단된 지점부터 다시 실행을 계속합니다.
   - `yield` 문은 여러 개 사용할 수 있으며, 각 `yield`마다 값을 생성합니다.

예를 들어, 다음과 같은 제너레이터 함수를 살펴보겠습니다:

In [4]:
def count_up_to(n):
    i = 1
    while i <= n:
        yield i
        i += 1

- 이 함수는 1부터 `n`까지의 숫자를 차례대로 생성하는 제너레이터입니다.
- `yield i`는 현재 값 `i`를 반환하고, 함수의 실행을 일시 중단합니다.
- `next()` 함수를 호출할 때마다 제너레이터는 다음 숫자를 생성합니다.

제너레이터 객체를 사용하는 방법은 다음과 같습니다:

In [6]:
gen = count_up_to(5)
print(next(gen))  # 1
print(next(gen))  # 2
print(next(gen))  # 3
print(next(gen))  # 4
print(next(gen))  # 5

1
2
3
4
5


- `count_up_to(5)`는 제너레이터 객체를 반환합니다.
- `next(gen)`을 호출할 때마다 제너레이터는 다음 숫자를 생성하고 반환합니다.
- 제너레이터가 모든 값을 생성한 후에 `next()`를 호출하면 `StopIteration` 예외가 발생합니다.

제너레이터는 대량의 데이터를 처리할 때 메모리 효율적이며, 필요한 값만 생성하므로 리소스를 절약할 수 있습니다. 또한, 제너레이터는 무한 시퀀스를 생성할 수도 있습니다.

### 2. Scrapy 프로젝트4

#### 1. 프로젝트 생성 및 설정

1. **프로젝트 생성**
   - 새 Scrapy 프로젝트를 생성합니다.
     ```bash
     scrapy startproject daveleefun_project4
     ```
     - `daveleefun_project4`라는 이름의 새 Scrapy 프로젝트가 생성됩니다.

2. **프로젝트 디렉토리로 이동**
   ```bash
   cd daveleefun_project4
   ```

3. **Spider 생성**
   ```bash
   scrapy genspider multiple_webs davelee-fun.github.io
   ```
   - `davelee-fun.github.io` 사이트를 크롤링할 `multiple_webs` Spider를 생성합니다.

#### 2. settings.py 수정

- 유사한 사이트 크롤링을 지원하기 위해 `settings.py`에 다음 설정을 추가합니다.
  ```python
  DUPEFILTER_CLASS = 'scrapy.dupefilters.BaseDupeFilter'
  ```
  - 이 설정은 Scrapy가 동일한 사이트에 대한 중복 크롤링을 허용하도록 합니다.

#### 3. items.py 수정

- 크롤링할 데이터의 구조를 정의합니다.
  ```python
  import scrapy

  class DaveleefunItem(scrapy.Item):
      # 예를 들어 페이지의 제목을 저장하기 위한 필드
      title = scrapy.Field()
  ```

#### 4. 여러 페이지 크롤링 로직 구현

- `multiple_webs.py` Spider 파일에 여러 페이지를 크롤링하는 로직을 추가합니다.
```python
import scrapy
from daveleefun_project4.items import DaveleefunItem

class MultipleWebsSpider(scrapy.Spider):
    name = 'multiple_webs'
    allowed_domains = ['davelee-fun.github.io']

    def start_requests(self):
        urls = ['http://davelee-fun.github.io/']
        urls.extend([f'https://davelee-fun.github.io/page{i}' for i in range(2, 7)])
        for url in urls:
            yield scrapy.Request(url, self.parse)

    def parse(self, response):
        # 예를 들어, 페이지의 제목을 추출하고 저장합니다.
        titles = response.css('h4.card-text::text').getall()
        for title in titles:
            item = DaveleefunItem()
            item['title'] = title
            yield item
```
  - 여기서 `start_requests` 메소드에서 초기 URL과 추가 페이지 URL들을 정의합니다.
  - `parse` 메소드에서는 실제 크롤링 로직을 구현하고, 추출된 데이터를 `DaveleefunItem` 객체에 저장한 후 `yield`를 통해 반환합니다.

#### 5. 크롤링 실행 및 결과 확인

- 크롤링을 실행하고 결과를 확인합니다.
  ```bash
  scrapy crawl multiple_webs -o multiple_webs_item.json
  ```
  - `-o` 옵션으로 출력 파일을 지정합니다. 여기서는 `multiple_webs_item.json` 파일에 크롤링 결과를 JSON 형식으로 저장합니다.


### 3. Scrapy 프로젝트5

#### 1. 프로젝트 생성 및 설정

1. **프로젝트 생성**
   - 새 Scrapy 프로젝트를 생성합니다.
     ```bash
     scrapy startproject daveleefun_project5
     ```
     - `daveleefun_project5`라는 이름의 새 Scrapy 프로젝트가 생성됩니다.

2. **프로젝트 디렉토리로 이동**
   ```bash
   cd daveleefun_project5
   ```

3. **Spider 생성**
   ```bash
   scrapy genspider multiple_webs davelee-fun.github.io
   ```

#### 2. items.py 수정

- 크롤링할 데이터의 구조를 정의합니다.
  ```python
  import scrapy

  class DaveleefunItem(scrapy.Item):
      title = scrapy.Field()
  ```

#### 3. 여러 페이지 크롤링 로직 구현

- Spider 파일에 여러 페이지를 크롤링하는 로직을 추가합니다.
```python
import scrapy
from daveleefun_project5.items import DaveleefunItem

class MultipleWebsSpider(scrapy.Spider):
    name = 'multiple_webs'
    allowed_domains = ['davelee-fun.github.io']

    def start_requests(self):
        urls = ['http://davelee-fun.github.io/']
        urls.extend([f'https://davelee-fun.github.io/page{i}' for i in range(2, 7)])
        for url in urls:
            yield scrapy.Request(url, self.parse)

    def parse(self, response):
        # 예를 들어, 페이지의 제목을 추출하고 저장합니다.
        titles = response.css('h4.card-text::text').getall()
        for title in titles:
            item = DaveleefunItem()
            item['title'] = title
            yield item
```

#### 4. pipelines.py 데이터 전처리 추가

- `pipelines.py`에 데이터 전처리 로직을 추가합니다.
  ```python
  class CleanTitlePipeline:
      def process_item(self, item, spider):
          # "상품명: " 접두어 제거
          item['title'] = item['title'].replace("상품명: ", "")
          # 불필요한 개행 문자 제거
          item['title'] = item['title'].strip()
          return item
  ```

- `settings.py`에서 `CleanTitlePipeline`을 활성화합니다.
  ```python
  ITEM_PIPELINES = {
      'daveleefun_project5.pipelines.CleanTitlePipeline': 300,
  }
  ```

#### 5. 크롤링 실행 및 결과 확인

- 크롤링을 실행하고 결과를 확인합니다.
  ```bash
  scrapy crawl multiple_webs -o multiple_webs_item.json
  ```

<div class="alert alert-block" style="border: 1px solid #455A64;background-color:#ECEFF1;">
본 자료 및 영상 컨텐츠는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하는 것을 금지합니다. 특히 자료에 대해서는 저작권법을 엄격하게 적용하겠습니다.
</div>